In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp random

# Random POVM's

In [ ]:
#export
import numpy as np
import qutip as qt
import scipy as sc

from qbism.povm import *

Here we follow the work of Heinosaari, Jivulescu, and Nechita as explained in this [paper](https://arxiv.org/abs/1902.04751v1) and in this [blog post](https://ion.nechita.net/2019/02/14/random-quantum-measurements/).

We want to generate a Haar randomly distributed POVM parameterized by $d$, the dimensionality of the Hilbert space, $k$, the number of outcomes, and $n$, the "environment parameter," which controls the mixedness of the POVM effects.

We begin by generating $k$ $d \times n$ "Ginibre matrices," which have "independent, identically distributed complex Gaussian entries" with variance $\frac{1}{2}$. 

In [ ]:
#export
def random_ginibre(n, m, real=False):
	return (np.random.randn(n, m) + 1j*np.random.randn(n,m))/np.sqrt(2) if not real else np.random.randn(n, m)/np.sqrt(2)

From these matrices $G$, we can form "Wishart" matrices: $W = GG^{\dagger}$. These will be random positive semidefinite matrices "of typical rank $min(d,n)$". 

So we have $k$ Wishart matrices: $W_{i}$. In order to get a POVM, we need to "divide" each $W_{i}$ by their sum:

$$A_{i} = S^{-\frac{1}{2}}W_{i}S^{-\frac{1}{2}}$$

Where $S = \sum_{j} W_{j}$.

Notice that to divide by the matrix sum, we multiply from the left and the right by the half-inverse of $S$. 

Finally, we also use the same algorithm to generate real-valued POVM's.

In [ ]:
#export
def random_haar_povm(d, k=None, n=1, real=False):
    r"""
    Generates a Haar distributed random POVM for a Hilbert space of dimension $d$, with $k$ elements, and with "mixedness" $n$.

    $n$ must satisfy $d \leq kn$, and defaults to $n=1$, giving rank-1 POVM elements.

    $k$ defaults to $d^2$ if complex, $\frac{d(d+1)}{2}$ if real.
    """
    k = k if type(k) != type(None) else (d**2 if not real else int(d*(d+1)/2))
    povm = np.zeros((k, d, d), dtype=np.complex128) if not real else np.zeros((k, d, d))
    S = np.zeros(d, dtype=np.complex128) if not real else np.zeros(d)
    for i in range(k):
        Xi = random_ginibre(d, n, real=real)
        Wi = Xi @ Xi.conjugate().T
        povm[i, :, :] = Wi
        S = S + Wi
    S = sc.linalg.fractional_matrix_power(S, -1/2)
    for i in range(k):
        Wi = np.squeeze(povm[i, :, :])
        povm[i, :, :] = S @ Wi @ S
    return [qt.Qobj(e) for e in povm]

Let's test it out:


In [ ]:
d = 3
rho = qt.rand_dm(d)
povm = random_haar_povm(d)
assert np.allclose(qt.identity(d), sum(povm))
assert np.allclose(rho, probs_dm(dm_probs(rho, povm), povm))

We can also generate randomly distributed POVM elements themselves:

In [ ]:
#export
def random_haar_effect(d, k=None, n=1, real=False):
    r"""
    Generates a Haar distributed random POVM effect of Hilbert space dimension $d$, as if it were part of a POVM of $k$ elements, with mixedness $n$. 
    """
    k = k if type(k) != type(None) else (d**2 if not real else int(d*(d+1)/2))
    X = random_ginibre(d, n, real=real)
    W = X @ X.conjugate().T
    Y = random_ginibre(d, (k-1)*n, real=real)
    S = W + Y @ Y.conjugate().T
    S = sc.linalg.fractional_matrix_power(S, -1/2)
    return qt.Qobj(S @ W @ S.conjugate().T)